<a href="https://colab.research.google.com/github/mengyuest/project-dcmk/blob/main/padm_proj_group4_task5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Teammates:

* Kayla Holman
* Lisa R Peng
* Will Parker
* Yue Meng


# Timeline

## Week 9 - Project Skeleton (Due: 4pm, Oct. 30, Friday)
1. Completed the required reading
2. Provide a skeleton Jupyter notebook
3. Outlining the main ideas of each topic in markup cells
4. Commented code cells which include function stubs for the capabilities to be demonstrated



## Week 12 - First Draft (Due: 4pm, Nov. 20, Friday)
1. Including detailed walkthroughs of the ideas involved in each topic
2. Working code complementing each section.
    - Implementation of main functions
    - Explanations about the functions


## Week 15 - Final Submission (4pm, Dec. 9, Wednesday)
A final draft incorporating feedback from the TAs

# Problem Background
YOU are an engineer hired by the Generic Space Empire for a project. You have been assigned to a moon
space station, the center of controversy throughout the galaxy.
Maintenance personnel have noticed that several workstations have been acting up, but have assured the
engineering team that “uh, everything’s perfectly all right now”. You, however, suspect something
sinister afoot. Could it be sabotage from the Underdog Rebels?
You know that a document for the space station was recently stolen (many Non-Copyrighted Alien spies
died), in which several attacks taking advantage of design flaws have been documented. Each attack
requires compromising a combination of components, distributed across the workstations.
While you can not dismantle the workstations to directly inspect the components, you can send a team of
diagnosis droids to make observations on the workstations to exactly determine the faulty components. As
you perform diagnosis on each workstation, you can update the odds on which type of attack will occur.
You must do this in a timely manner, so that you can escape before an attack if necessary, and warn the
Big Bad Guy Wearing Black if time permits.


##Temporal Planning and Reasoning
In this task, you will be asked to check whether you have time to warn the Big Bad Guy Wearing Black,
or whether you should just try to escape. You will be given duration information for traversals, diagnosis,
and possible additional actions you might take, as well as timing constraints you must meet in order to
escape on a shuttle or provide warnings. You will be asked to check for the feasibility of alternative plans,
providing a schedule for a feasible plan, and provide an explanation for each infeasible plan.

Inputs:
1. Set of possible actions
2. Set of events
3. Temporal constraints

Outputs:
1. Feasibility?
2. Conflict if infeasible
3. Schedule if feasible
4. Best temporally feasible plan


## Readings
Representing temporal relations, Allen’s temporal algebra and simple temporal networks; temporal consistency, scheduling and dynamic execution. 

Reading: 
* Dechter, Meiri and Pearl, “Temporal Constraint Networks,” – on
Stellar. 
* Optional Reading: Muscettola, Morris and Tsamardinos, “Reformulation Temporal Plans for Efficient
Execution,” – on Stellar.

# Code Section

1. First translate input to multiple constraint graphs
2. Then for each constraint graph:
    1. Check whether it's feasible (and point out where it causes conflict)
    2. Compute its minimal distance graph
    3. Use minimal distance graph to do scheduling
    4. Output the best feasible plan (I guess saving the Big Guy, and also in the shortest time?)

In [ ]:
# necessary data structures for APSP representation
class Node:
    pass


class Edge:
    pass


class Graph:
    pass
    

# Construct the Distance Graph
Given a list of actions, events, and constraints for a given constraint problem, builds the distance graph using the graph implementation defined above. For the constraints given, the upper bounds of the intervals connect nodes in the forward direction, and lower bounds connect nodes in the backwards direction and are negated. 

In [ ]:
def build_distance_graphs(actions, events, constraints):
    '''
    Params:
      actions:
      events:
      constraints:
    Returns:
      An instance of the Graph class representing the distance graph for the given inputs
    '''
    raise NotImplementedError

# Checking Consistency of Distance Graph
Given the distance graph, we need to check if there are any negative cycles that will render the problem unsolvable. This can either be done after making the distance graph or after creating the d-graph, but to avoid longer runtimes when there is a consistency, we'll do the check before making the d-graph. This will be done using the Bellman-Ford algorithm.

In [ ]:
def is_consistent(graph):
    '''
    Params:
      graph: An instance of the Graph class representing the distance graph
    Returns:
      False if the graph has negative cycles, True otherwise
    '''
    raise NotImplementedError
    #check outputted dp of APSP for negative values along the diagonal entries d_ii
    #TODO(yue): another way to check consistency is to use Bellman-Ford algorithm
    #           which gives O(VE) time complexity, as suggested from 
    #           https://stackoverflow.com/questions/16898416/fastest-algorithm-to-detect-if-there-is-negative-cycle-in-a-graph



# Construct the APSP Complete Directed Graph 

Assuming that the provided scenario has been determined to be consistent, use the set of events, actions, and temporal constraints to construct a d-graph to solve the all-pairs-shortest-path problem. 


In [ ]:
def floyd_warshall_shortest_path(graph):
    raise NotImplementedError
    #this is Lisa: what's the diff between this fxn and build_apsp_graph?
    #TODO(Yue): there are two ways to setup APSP graph:
    #           1. Floyd-Warshall algorithm (O(V^3)) - fits dense graphs
    #           2. Johnson’s algorithm (O(V^2\log V + VE)) - fits sparse graphs
    #           it would be cool if we can adaptively route the build_apsp_graph
    #           function towards different handlers, based on graph sparsity


In [ ]:
def johnson_shortest_path(graph)：
    raise NotImplementedError


In [ ]:
def build_apsp_d_graph(graph):
    raise NotImplementedError

In [ ]:
def find_conflicts(graph):
    raise NotImplementedError


Use Solution by Decomposition to extract a path.

input: APSP d-graph

output: assignment as a list representing sequence of actions with time interval pairs

TODO: decide on human-readable output format, todo after we get a concrete idea of the data of the problem setup. 

In [ ]:


# assignment-related functions
def evaluate_assignment(assignment):
    raise NotImplementedError

# assignment-related functions
def assignment_to_string(assignment):
    raise NotImplementedError


In [ ]:
def main(actions, events, constraints):
    graphs = build_constraint_graphs(actions, events, constraints)
    assigment_list = []
    for i,graph in enumerate(graphs):
        if is_consistent(graph):
            apsp_graph = build_apsp_graph(graph)
            assignment = schedule_under_apsp_graph(apsp_graph)
            print("graph-%d is feasbile, and one greedy plan is %s"%(i, assignment))
            assigment_list.append(assignment)
        else:
            conflicts = find_conflicts(graph)
            print("graph-%d is infeasible, and the conflicts are found in %s"%(i, conflicts))

    if len(assignment_list)==0:
        print("No feasible plan can be found")
    else:
        best_assignment = sorted(assignment_list, key=lambda x:evaluate_assignment(x), reversed=True)[0]
        print("The best plan is %s"%(best_assignment))


#TODO do some tests
main([],[],[])

# Friday's assignment

#TODO data structure (Yue)
#TODO (describe for)
# 1. build_constraint_graphs (Kayla)
# 2. is_consistency (Kayla)
# 3. build_apsp_graph (Will)
# 4. schedule_under_apsp_graph (Lisa)


## Notes for "Temporal Constraint Networks"

Related papers:
1. (Allen's interval algebra) Allen, James F. "Maintaining knowledge about temporal intervals." Communications of the ACM 26.11 (1983): 832-843.
2. (Point algebra) Vilain, Marc B., and Henry A. Kautz. "Constraint propagation algorithms for temporal reasoning." Aaai. Vol. 86. 1986.
3. (Linear inequalities) Malik, Jitendra, and Thomas O. Binford. "Reasoning in Time and Space." IJCAI. Vol. 83. 1983.
4. (Linear inequalities) Valdes-Perez, Raul E. "Spatio-temporal reasoning and linear inequalities." (1986).
5. (Time map) Dean, Thomas L., and Drew V. McDermott. "Temporal data base management." Artificial intelligence 32.1 (1987): 1-55.


### TCSP Model
TCSP-*Temporal constraint satisfaction problem* is CSP that contains variables $\{X_1\cdots X_n\}$ and constraints where each constraint is a set of intervals $\{\cdots,I_k,\cdots\}=\{\cdots,[a_k,b_k],\cdots\}$

Two types of constraints:
1. unary: $(a_1\leq X_i\leq b_i)\vee \cdots \vee (a_n\leq X_i\leq b_n)$
2. binary: $(a_1\leq X_j-X_i\leq b_i)\vee \cdots \vee (a_n\leq X_j-X_i\leq b_n)$

Can treat all constraints as binary constraints by adding a reference point $X_0$

**Solution**: tuple $\mathcal{X}=\{x_1\cdots x_n\}$ satisifies all the constraints.

v is a **feasible value** for $X_i$ if exists a solution in which $X_i=v$.

**Minimal domain**: All feasible values of a variable.

Network is **consistent** if at least one solution exists.


Set operations ($T=\{I_1,\cdots I_l\}$, $S=\{J_1,\cdots J_m\}$): 
1. *union*: $T\cup S=\{I_1,\cdots,I_l,J_1,\cdots J_m\}$
2. *intersection*: $T\oplus U=\{K|K=I_i\cap J_j, \forall i,j\}$
3. *composition*: $T\otimes U=\{K|K=[a+c,b+d], \forall I_i=[a,b],\,J_l=[c,d]\}$

**Network level:**
1. **Partial order**: $T\subseteq S$ if all constraints meet $T_{ij}\subseteq S_{ij}$( for every interval $J\in S_{ij}$, there exist $I\in T_{ij}$, such that $I\subseteq J$ (which means $T_{ij}$ is tighter than $S_{ij}$)).
2. Two networks are **equivalent** if they represent the same solution set.
3. **Minimal network**: the one with respect to $\subseteq$ from all equivalent representations of a network.
4. A network is **decomposable** if every locally consistent assignment to any set of variables can be extended to a solution.

### Simple Temporal Problem (STP)
> Definition: all constraints specify a single interval.

**Distance graph**: Associate an STP with a directed edge-weighted graph $G_d=(V,E_d)$

Denote $d_ij$ the length of the shortest path from $i$ to $j$.

**Theorem 3.1** STP is consistent iff its distance graph has no negative cycles.

**Corollary 3.2.** For STP with graph $G_d$, two consistent scenarios are $S_1=(d_{01},\cdots,d_{0n})$ and $S_2=(-d_{10},\cdots,-d_{n0})$ which assign to each variable its latest and earliest possible time.

**A given STP can be specified by a complete directed graph, called d-graph, where each edge $i\to j$ is labeled by the shortest path length $d_{ij}$ in $G_d$.**


**Theorem 3.3** Any consistent STP is decomposable relative to the constraints in its d-graph.

**Corollary 3.4.** For STP with graph $G_d$, the set of feasible values for variable $X_i$ is $[-d_{i0}, d_{0i}]$

**Corollary 3.5.** For a STP $T$, the graph defined by $M_{ij}=\{[-d_{ji},d_{ij}]\}$ is the minimal network representation for $T$.

**d-graph of an STP can be constructed by applying Floyd-Warshall's all-pairs-shortest-paths algorithm to the distance graph. (it can also check consistency, by examining the sign of the diagonal elements $d_{ii}$)** Time complexity $O(n^3)$. Once the d-graph is ready, finding a solution requires $O(n^2)$ time.


**(TODO)Still not clear how to find a solution and why it's $O(n^2)$**


### The general TCSP
> Edges may be labeled by several intervals

Deciding consistency of TCSP is NP-Hard

One **naive** way to solve TCSP is to decompose it into several STPs, solve each, and combine the result.

The "Backtrack" algorithm (recursively Forward and Go-back) is in Fig-5

But this one too expensive- $O(n^3k^e)$ (n nodes, k intervals for each constraint)







### Paths consistency algorithms
> An approximation method, or say preprocessing, by finding path-consistent network

**Path-Consistency**

PC-1, similar to Montanari's path consistency algorithm (shown in Fig6)

**Path Consist Network?=>Minimal Network**: True for STP due to distributivity holds, but does not hold for TCSP

Path-consistent cannot detect inconsistency.

PC-2 (more efficient, Fig-8)

For discrete CSPs, PC-2 achieves path consistency in $O(n^3R^3)$, where $R$ is the maximum range of any constraint (in coarest units).

**Directional path consistency** - a weaker version of path consistency. (more efficient)


For TCSP, directional path consistency can be acheived by algorithm DPC (single-pass comparing to PC-1).

induced-graph

DPC Complexity: $O(nW^*(d)^2)$, where $W^*(d)$ is the maximum number of parents a node has in the induced-graph.




### Network-based algorithms
> The topological features of the constraint network may guide to select efficient solution methods, lower complexity than naive backtracking.


*series-parallel networks* (equiv. to *regualr width-2 networks*)


This network can yield a backtrack-free solution.



Decomposition into nonseparable components: First find the nonseparable components, then solve each of them independently. If all components are consistent, then the entire network is consistent, and minimal networks of components coincide with the overall minimal network.

### Summary
1. STP and general TCSP (multi-interval constraint per node)
2. STP is solved in polynomial time using FloydWarshall
3. TCSP is solved by decomposition to STP, but not an efficient algorithm.
4. Use path-consistency as preprocessing
5. Decomposing nonseparable components was found applicable to TCSP


# Notes for "Reformulating Temporal Plans For Efficient Execution"


Temporal Plan: <G,T,A>, where G is an STN, T is a set of tasks, and A is from a set of association  tuples defined on the nodes of G and on the tasks of T (e.g. "begin", "end")

